# Part I

### Reading an image

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import io

rice = io.imread('rice.jpg')
plt.imshow(rice, cmap='gray')
plt.show()

In [ ]:
print(rice.shape) # shaap of the image
print(rice.dtype) # data type of the image
print(rice) # how the image looks like

In [ ]:
mm = io.imread('mm.jpg')
plt.imshow(mm)
plt.show()

In [ ]:
print(mm.shape)
print(mm.dtype)
print(mm)

In [ ]:
mm_gray = io.imread('mm.jpg', as_gray=True) # read it in grayscale
plt.imshow(mm_gray, cmap=plt.cm.gray) # show it in grayscale
plt.show()

In [ ]:
print(mm_gray.shape)
print(mm_gray.dtype)
print(mm_gray)

### Change the brightness

In [ ]:
rice_bright = rice + 20
plt.imshow(rice)
plt.show()
plt.imshow(rice_bright)
plt.show()
rice_brighter = rice + 100 # what's wrong with this image
plt.imshow(rice_brighter)
plt.show()

### Integer can overflow !

In [ ]:
from skimage import img_as_float
rice_float = img_as_float(rice) # read it in float type!
rice_float_15 = rice_float * 1.5
plt.imshow(rice)
plt.show()
plt.imshow(rice_float_15, vmax=1) # set vmax=1
plt.show()

### Image addition

In [ ]:
from skimage import data
camera = data.camera()
plt.imshow(camera)
plt.show()
grass = data.grass()
plt.imshow(grass)
plt.show()
added = img_as_float(grass) + img_as_float(camera)
plt.imshow(added)
plt.show()

### Histogram

In [ ]:
import skdemo

skdemo.imshow_with_histogram(rice)

cat = data.cat()
skdemo.imshow_with_histogram(cat)

In [ ]:
from skimage import exposure

skdemo.imshow_with_histogram(rice)

high_contrast_rice = exposure.rescale_intensity(rice, in_range=(100, 180))
print(high_contrast_rice.min(), high_contrast_rice.max())
skdemo.imshow_with_histogram(high_contrast_rice, vmin=0, vmax=255)

low_contrast_rice = exposure.rescale_intensity(rice, out_range=(100, 180))
print(low_contrast_rice.min(), low_contrast_rice.max())
skdemo.imshow_with_histogram(low_contrast_rice, vmin=0, vmax=255)

In [ ]:
skdemo.imshow_with_histogram(rice)

rice_equalized = exposure.equalize_hist(rice)
skdemo.imshow_with_histogram(rice_equalized)

In [ ]:
# E1.1
...

#E1.2
...

#E1.3
...

#E1.4
...

#E1.5
...

#E1.6
...

# Part II

### Binarization

In [ ]:
rice_float = img_as_float(rice)
rice_binarization = rice_float > 0.56
plt.imshow(rice)
plt.show()

plt.imshow(rice_binarization)
plt.show()

### Otsu's method

In [ ]:
from skimage import filters

otsu_thresh = filters.threshold_otsu(rice)
print(otsu_thresh)
otsu_binary = rice > otsu_thresh
plt.imshow(otsu_binary)
plt.show()

### Local thresholding

In [ ]:
block_size = 55
local_thresh = filters.threshold_local(rice, block_size, offset=-20) # what is offset?
local_binary = rice > local_thresh
plt.imshow(local_binary)
plt.show()

### Morphology

In [ ]:
from skimage.morphology import erosion, dilation, opening, closing, square

img = img_as_float(rice)

bg_erosion = erosion(img, square(7))
plt.imshow(bg_erosion)
plt.show()

bg_dilation = dilation(img, square(7))
plt.imshow(bg_dilation)
plt.show()

bg_opening = opening(img, square(7))
plt.imshow(bg_opening)
plt.show()

bg_closing = closing(img, square(7))
plt.imshow(bg_closing)
plt.show()

bg = filters.gaussian(bg_erosion, sigma=7) # How bg looks like
plt.imshow(bg)
plt.show()

img_no_bg = img - bg
plt.imshow(img_no_bg)
plt.show()

### Color space

In [ ]:
from skimage import color

mm = io.imread('mm.jpg')
plt.imshow(mm)
plt.show()

plt.imshow(mm[:, :, 0])
plt.show()

plt.imshow(mm[:, :, 1])
plt.show()

plt.imshow(mm[:, :, 2])
plt.show()

In [ ]:
img_red = mm[:, :, 0] > 0.7
plt.imshow(img_red)
plt.show()

In [ ]:
mm_hsv = color.rgb2hsv(mm)
plt.imshow(mm_hsv)
plt.show()

plt.imshow(mm_hsv[:, :, 0])
plt.show()

plt.imshow(mm_hsv[:, :, 1])
plt.show()

plt.imshow(mm_hsv[:, :, 2])
plt.show()

In [ ]:
mm_hue = mm_hsv[:, :, 0]

img = (mm_hue > 0.3) & (mm_hue < 0.35)
plt.imshow(img)
plt.show()

cropped_green = img_as_float(mm) * np.stack((img, )*3, axis=-1)
plt.imshow(cropped_green)
plt.show()

bg = (mm_hue > 0.112) & (mm_hue < 0.15) # background
plt.imshow(bg)
plt.show()

img_com = 1.0 - bg # complementary
plt.imshow(img_com)
plt.show()

cropped = (img_as_float(mm) * np.stack((img_com, )*3, axis=-1))

plt.imshow(cropped)
plt.show()

In [ ]:
#E2
...

# Part III

### Counting by contour

In [ ]:
from skimage.measure import find_contours
contour_image = img_as_float(local_binary.copy())
contours = find_contours(contour_image, fully_connected='high')

fig, ax = plt.subplots()
ax.imshow(contour_image, cmap='gray')

for contour in contours:
    ax.plot(contour[:, 1], contour[:, 0], linewidth=2, color='red')

print("Number of grains", len(contours))

ax.axis('image')
ax.set_xticks([])
ax.set_yticks([])
plt.show()

### Counting by circles

In [ ]:
import scipy.ndimage as nd
from skimage.draw import circle_perimeter
from skimage.feature import canny
from skimage.transform import hough_circle, hough_circle_peaks

mm_gray = color.rgb2gray(mm)
edges = canny(mm_gray)
hough_radii = np.arange(15, 20)
hough_res = hough_circle(edges, hough_radii)
accums, cx, cy, radii = hough_circle_peaks(hough_res, hough_radii, 
                                           min_xdistance=15,
                                           min_ydistance=15,
                                           threshold=0.44)

fig, ax = plt.subplots(ncols=3, nrows=1, figsize=(12, 6))

image = mm.copy()
blank = np.zeros((mm.shape)).astype(np.uint8)
print(f'Number of mm: {len(radii)}')
for center_y, center_x, radius in zip(cy, cx, radii):
    circy, circx = circle_perimeter(center_y, center_x, radius, shape=mm.shape)
    blank[circy, circx] = (255, 255, 255)
    
filled_mm = nd.morphology.binary_fill_holes(blank[:, :, 0]).astype(int)

ax[0].imshow(mm)
ax[1].imshow(blank[:, :, 0], cmap=plt.cm.gray)
ax[2].imshow(filled_mm, cmap=plt.cm.gray)
plt.show()

In [ ]:
#E3
...